In [0]:
# Upload the files 'posts.csv', 'users.csv', and 'views.csv' on google drive and save these files in a folder 'database' if you want to use google drive and skip the next block
from google.colab import drive
drive.mount('/content/gdrive')
path = 'gdrive/My Drive/database/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Upload the files 'posts.csv', 'users.csv', and 'views.csv' on google colab for the current session and skip the previous block
path = ''

In [0]:
#------------------------------------------------------#
# CONTENT BASED FILTERING TO RECOMMEND POSTS FOR USERS #
#------------------------------------------------------#

In [0]:
# note that this is very similar to post-based collaborative filtering

In [0]:
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [0]:
posts = pd.read_csv(path+'posts.csv')
users = pd.read_csv(path+'users.csv')
views = pd.read_csv(path+'views.csv')

print("no. of posts {0}".format(len(posts)))
print("no. of users {0}".format(len(users)))
print("no. of views {0}".format(len(views)))

no. of posts 493
no. of users 118
no. of views 1449


In [0]:
posts.head(5)

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [0]:
users.head(5)

,_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [0]:
views.head(5)

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


In [0]:
print("number of unique categories: {0}".format(len(posts['category'].unique())))
print("number of users who have seen at least one post: {0}".format(len(views['user_id'].unique())))

number of unique categories: 232
number of users who have seen at least one post: 118


In [0]:
tfidf = TfidfVectorizer(stop_words='english')
posts['category'] = posts['category'].fillna('')
tfidf_matrix = tfidf.fit_transform(posts['category'])
tfidf_matrix.shape

(493, 266)

In [0]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(493, 493)

In [0]:
indices = pd.Series(posts.index, index=posts['_id']).drop_duplicates()
indices[:5]

_id
5d62abaa65218653a132c956    0
5d6d39567fa40e1417a4931c    1
5d7d23315720533e15c3b1ee    2
5d7d405e5720533e15c3b1f3    3
5d80dfbc6c53455f896e600e    4
dtype: int64

In [0]:
dic_post_user = defaultdict(list)
for i in range(len(views)):
  row = views.iloc[i]
  dic_post_user[row['user_id']].append(row['post_id'])

In [0]:
def get_recommendations(postid, cosine_sim=cosine_sim):
  idx = indices[postid]
  
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  
  selected = []
  for i in sim_scores:
    if i[1]>0.5:
      selected.append(i)
  selected = selected[1:11]

  post_indices = [i[0] for i in selected]
  return posts['category'].iloc[post_indices]

In [0]:
def post_type(user_posts):
  pref = user_posts[' post_type'].value_counts(ascending = False)
  return pref

In [0]:
def recommend(user_id):
  view_user = views[views['user_id'] == user_id]
  view_user = view_user.sort_values(by = ['timestamp'], ascending = False)

  user_posts = posts[posts['_id'].isin(view_user['post_id'])]
  user_posts = pd.merge(user_posts, view_user, left_on = '_id', right_on = 'post_id')
  user_posts = user_posts.sort_values(by = ['timestamp'], ascending = False)
  
  pref = post_type(user_posts)
  
  posts_rec = pd.Series([],dtype=object)
  for ind, value in view_user['post_id'].items():
    try:
      post = get_recommendations(value)
      posts_rec = posts_rec.append(post)
    except:
      continue
  posts_rec = posts_rec.loc[~posts_rec.index.duplicated(keep = 'first')]
  posts_rec = posts.iloc[posts_rec.index]
  posts_rec['preference'] = posts_rec[' post_type'].apply(lambda x: pref.get(key = x))
  posts_rec = posts_rec.fillna(0)
  
  start = 0
  end = 0
  for i in range(len(posts_rec)-1):
    a = posts_rec.iloc[i]
    b = posts_rec.iloc[i+1]
    if a['category'] ==  b['category']:
      end+=1
    else:
      posts_rec[start:end+1] = posts_rec[start:end+1].sort_values(by = ['preference'], ascending = False)
      start = end + 1
      end = end + 1

  print('_'*100)
  print('_'*100)
  print()
  print("POSTS ALREADY SEEN BY USER IN ORDER OF TIMESTAMP\n")
  for i in range(len(user_posts)):
    post = user_posts.iloc[i]
    print("Title: {}".format(post['title']))
    print("Category: {}".format(post['category']))
    print("Post Type: {}".format(post[' post_type']))
    print()
  
  print('_'*100)
  print('_'*100)
  print()
  print("POSTS RECOMMENDED FOR USER\n")
  rec_count = 0
  for i in range(len(posts_rec)):
    post = posts_rec.iloc[i]
    if post['_id'] not in dic_post_user[user_id]:
      print("Title: {}".format(post['title']))
      print("Category: {}".format(post['category']))
      print("Post Type: {}".format(post[' post_type']))
      print()
      rec_count += 1
      if rec_count >= max(10,len(posts_rec)/3):
        break

In [0]:
user = input("Enter the user id to whom you want to recommend : ")
recommend(user)

Enter the user id to whom you want to recommend : 5ed0defa76027d35905cc2de
____________________________________________________________________________________________________
____________________________________________________________________________________________________

POSTS ALREADY SEEN BY USER IN ORDER OF TIMESTAMP

Title: She got a new hairstyle! Photo by Chirag
Category: Photography
Post Type: artwork

Title: AWS services and how to launch OS on AWS Cloud
Category: Computer Technology|Cloud Computing
Post Type: blog

Title: Configure Docker with Django; PostgreSQL; Pg-admin & Elasticsearch
Category: Computer Technology|Computer Application|Information Technology
Post Type: blog

Title: Energy.
Category: Calligraphy|Typography|Pen and ink
Post Type: artwork

Title: Face Recognition using Transfer Learning.
Category: Science; Technology
Post Type: skill

____________________________________________________________________________________________________
______________________

In [0]:
#-----------------------------------------------------------------#
# POST-BASED COLLABORATIVE FILTERING TO RECOMMEND POSTS FOR USERS #
#-----------------------------------------------------------------#

In [0]:
get_sim_post = input("Enter the post id to which you want similar posts : ")
row = posts[posts['_id'] == get_sim_post]
print('_'*100)
print('_'*100)
print("GIVEN POST\n")
print("Title: {}".format(row['title'].values[0]))
print("Category: {}".format(row['category'].values[0]))
print("Post Type: {}".format(row[' post_type'].values[0]))
print('_'*100)
print('_'*100)
print("SIMILAR POSTS\n")
sim_posts_input = get_recommendations(get_sim_post)
sim_posts_input = sim_posts_input.loc[~sim_posts_input.index.duplicated(keep = 'first')]
sim_posts_input = posts.iloc[sim_posts_input.index]
for i in range(len(sim_posts_input)):
  post = sim_posts_input.iloc[i]
  if post['_id'] != get_sim_post:
    print("Title: {}".format(post['title']))
    print("Category: {}".format(post['category']))
    print("Post Type: {}".format(post[' post_type']))
    print()

Enter the post id to which you want similar posts : 5ed1ff0276027d35905cc60d
____________________________________________________________________________________________________
____________________________________________________________________________________________________
GIVEN POST

Title: AWS services and how to launch OS on AWS Cloud
Category: Computer Technology|Cloud Computing
Post Type: blog
____________________________________________________________________________________________________
____________________________________________________________________________________________________
SIMILAR POSTS

Title: Understanding Cloud Computing(AWS)
Category: Computer Technology|Cloud Computing
Post Type: blog

Title: Cloud Computing
Category: Computer Technology|Computation|Computer Application|Cloud Computing
Post Type: blog

Title: 8 Best Machine Learning Courses for 2020
Category: Computer Technology|Cloud Computing|Artificial Intelligence|Information Technology|Programming

In [0]:
#-----------------------------------------------------------------#
# USER-BASED COLLABORATIVE FILTERING TO RECOMMEND POSTS FOR USERS #
#-----------------------------------------------------------------#

In [0]:
view_matrix = pd.crosstab(views.user_id, views.post_id)
view_matrix.replace(0, view_matrix.mean(axis=0), inplace= True)
view_matrix.head()

post_id,5d62abaa65218653a132c956,5d6d39567fa40e1417a4931c,5d7d23315720533e15c3b1ee,5d7d405e5720533e15c3b1f3,5d80dfbc6c53455f896e600e,5d80e7c16c53455f896e6014,5d80ecfd6c53455f896e601a,5d81323a6c53455f896e6044,5d9b3514979d5962253c2f90,5d9b950768671220a1b2b153,5da745b6019399436815c4cd,5dada695610ba040fbfdf585,5dada734610ba040fbfdf588,5dada798610ba040fbfdf58b,5dada82f610ba040fbfdf58e,5dada89b610ba040fbfdf591,5dadf6ae610ba040fbfdf5c8,5db2a7d099cbb90e4339c734,5db2b91a99cbb90e4339c749,5dbc622a99cbb90e4339c7f6,5dbc631f99cbb90e4339c7fd,5dc065ca24b883670268772f,5dd1751db802e41ed198b680,5dde6a91369b28584ecca156,5ddeb6e80eb5e25a8a07f065,5de179d80eb5e25a8a07f079,5de7971b8eab6401affbb137,5de8d73249e8203ff9219a74,5dea816a42a8854bf6eaba89,5dee9b5042a8854bf6eabaaf,5df64ad1cc709107827fb3dd,5e04fb252a37d20505da2a2b,5e1029f22a37d20505da2a79,5e27002d2a37d20505da2c13,5e281cf0025ae20886de2050,5e2867f6025ae20886de205a,5e294fd1c85ab714a7da6643,5e2d36ebc85ab714a7da66d1,5e2d3fbfc85ab714a7da66d5,5e2d447dc85ab714a7da66d7,...,5ecf23b87023451e66223944,5ecf818376027d35905cbf03,5ecf96e876027d35905cbf46,5ecfa0ca76027d35905cbf57,5ecfafcc76027d35905cbfb0,5ecfb35c76027d35905cbfbc,5ecfffb976027d35905cc0e9,5ed0007a76027d35905cc0ea,5ed09ad076027d35905cc15f,5ed0e20776027d35905cc2fe,5ed0e31a76027d35905cc302,5ed0e65976027d35905cc356,5ed0f0c576027d35905cc3c7,5ed13d2876027d35905cc4c2,5ed141aa76027d35905cc4c9,5ed175f176027d35905cc554,5ed175f476027d35905cc562,5ed1ff0276027d35905cc60d,5ed229d976027d35905cc666,5ed2378276027d35905cc6b5,5ed237d676027d35905cc6bd,5ed2383776027d35905cc6ca,5ed238d376027d35905cc6e5,5ed23a6376027d35905cc73b,5ed23b0776027d35905cc746,5ed23b6276027d35905cc74f,5ed23bb876027d35905cc759,5ed23c1876027d35905cc761,5ed23c8676027d35905cc780,5ed23cb876027d35905cc788,5ed23cf876027d35905cc790,5ed23d4276027d35905cc798,5ed23e4d76027d35905cc7b8,5ed2502b76027d35905cc7db,5ed3476576027d35905cca1d,5ed3791976027d35905ccab6,5ed3820f76027d35905ccac8,5ed3ef4cbd514d602c1530f2,5ed415c6bd514d602c15312d,5ed4cbadbd514d602c1531a6
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5d60098a653a331687083238,1.000000,1.000000,0.016949,0.025424,0.016949,0.016949,0.008475,0.008475,0.008475,0.008475,0.008475,0.016949,0.008475,0.008475,0.008475,0.008475,0.008475,0.008475,0.025424,0.008475,1.000000,1.000000,0.008475,0.008475,1.000000,0.008475,0.008475,0.016949,0.008475,0.025424,0.025424,0.008475,1.000000,0.008475,0.016949,0.008475,1.000000,1.000000,0.016949,0.033898,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.016949,1.000000,0.008475,1.000000,1.000000,0.084746,0.033898,0.025424,1.000000,0.059322,0.033898,0.008475,1.000000,0.016949,0.008475,0.008475,0.016949,0.008475,0.008475,0.008475,0.008475,0.008475,0.008475,0.008475,0.033898,0.008475,0.008475,0.008475,0.033898,0.016949,0.016949,1.000000,0.008475,0.016949,0.008475
5d610ae1653a331687083239,0.008475,0.008475,0.016949,0.025424,1.000000,0.016949,0.008475,0.008475,0.008475,0.008475,0.008475,0.016949,0.008475,0.008475,0.008475,0.008475,0.008475,0.008475,1.000000,0.008475,0.008475,0.008475,0.008475,0.008475,0.016949,0.008475,0.008475,0.016949,0.008475,0.025424,0.025424,0.008475,0.016949,0.008475,0.016949,0.008475,0.008475,1.000000,1.000000,1.000000,...,0.033898,0.042373,0.050847,0.033898,0.025424,0.016949,0.008475,0.008475,0.110169,0.016949,1.000000,0.033898,0.025424,1.000000,0.059322,0.033898,0.008475,0.101695,0.016949,0.008475,0.008475,0.016949,0.008475,0.008475,0.008475,0.008475,0.008475,0.008475,0.008475,0.033898,0.008475,0.008475,0.008475,0.033898,0.016949,0.016949,0.033898,0.008475,0.016949,0.008475
5d618359fc5fcf3bdd9a0910,0.008475,0.008475,0.016949,0.025424,0.016949,1.000000,0.008475,0.008475,0.008475,0.008475,0.008475,0.016949,0.008475,0.008475,0.008475,0.008475,0.008475,0.008475,0.025424,0.008475,0.008475,0.008475,0.008475,0.008475,0.016949,0.008475,0.008475,0.016949,0.008475,0.025424,0.025424,0.008475,0.016949,0.008475,0.016949,0.008475,0.008475,0.042373,0.016949,0.033898,...,0.033898,1.0

In [0]:
cosine = cosine_similarity(view_matrix)
np.fill_diagonal(cosine, 0 )
similarity_with_users = pd.DataFrame(cosine, index = view_matrix.index)
similarity_with_users.columns = users._id
similarity_with_users.head()

_id,5d60098a653a331687083238,5d610ae1653a331687083239,5d618359fc5fcf3bdd9a0910,5d6d2bb87fa40e1417a49315,5d7c994d5720533e15c3b1e9,5d90fc1a641ef364939fb6cc,5d9a2982979d5962253c2f81,5de50d768eab6401affbb135,5ded443342a8854bf6eaba9a,5deeef6142a8854bf6eabab9,5defd51362624b0135ea9fd2,5df11ee1ee4bb5252b4f5344,5df20f1fee4bb5252b4f5351,5df24eeaee4bb5252b4f5366,5df3f8f2ee4bb5252b4f5393,5df49b32cc709107827fb3c7,5e0d5ce12a37d20505da2a5d,5e1ef04c2a37d20505da2b8b,5e26bb482a37d20505da2c10,5e3563348d344822fed4d13a,5e35a5ed8d344822fed4d13e,5e365e758d344822fed4d144,5e37b16ceab55d319938a6f8,5e4195c3eab55d319938a82c,5e4ce251f5561b1994c8e40d,5e5855ced701ab08af792b51,5e5af599d701ab08af792b63,5e5b4511d701ab08af792b6d,5e5dfbbefbc8805f69e02c91,5e783375cfc8b713f5ac7c5b,5e78ce84cfc8b713f5ac7cee,5e7b30a6cfc8b713f5ac7d95,5e7cf05bcfc8b713f5ac7db7,5e7dde87a3258347b42f2108,5e7de336a3258347b42f210e,5e7def80a3258347b42f2124,5e7f4ad1a3258347b42f2155,5e822b86a3258347b42f2360,5e8309c4a3258347b42f23f4,5e840a75a3258347b42f2437,...,5eb4f73910426255a7aaa0cc,5eb508ae10426255a7aaa12c,5eb7aeccfd92f539c465ddd7,5ebc2ec4514aab59896bcc84,5ebcf2f2514aab59896bcd00,5ec11abc74f7660d73aa0fbd,5ec2204374f7660d73aa100f,5ec3ba5374f7660d73aa1201,5ec553e2f2781131cc7e5138,5ecb979eeaff6b0c3a58a4f0,5ecd5a5c7023451e662235c1,5ecdf7397023451e66223784,5ece02427023451e66223861,5ecf6e6576027d35905cbeec,5ecfa94676027d35905cbf78,5ecfaa0076027d35905cbf83,5ecfbcb376027d35905cbfe5,5ecfdaa976027d35905cc02c,5ed09e8576027d35905cc164,5ed0bfc176027d35905cc1f8,5ed0cfc676027d35905cc26a,5ed0d9bc76027d35905cc2c2,5ed0defa76027d35905cc2de,5ed0dfec76027d35905cc2ec,5ed0fc0976027d35905cc448,5ed12ae476027d35905cc4a1,5ed13bb076027d35905cc4ba,5ed142cd76027d35905cc4cd,5ed232b976027d35905cc689,5ed237ee76027d35905cc6c5,5ed2399b76027d35905cc70f,5ed24d5a76027d35905cc7d7,5ed2672576027d35905cc7f6,5ed2edf176027d35905cc961,5ed350ed76027d35905cca2c,5ed35aa376027d35905cca67,5ed3737976027d35905ccaa2,5ed3748576027d35905ccaab,5ed3cbcbbd514d602c1530a7,5ed3e898bd514d602c1530de
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5d60098a653a331687083238,0.000000,0.424553,0.226683,0.325020,0.348116,0.388970,0.341544,0.286228,0.388970,0.298341,0.272349,0.389865,0.132890,0.387124,0.296121,0.263768,0.342224,0.238974,0.206862,0.302448,0.226226,0.304589,0.170814,0.282231,0.436296,0.437597,0.356488,0.384762,0.396477,0.340192,0.388970,0.384750,0.268953,0.389412,0.259291,0.342151,0.256829,0.379789,0.285498,0.389412,...,0.262508,0.376532,0.341935,0.437597,0.377807,0.342107,0.304396,0.306934,0.435315,0.184359,0.389412,0.348649,0.375019,0.308148,0.390803,0.391786,0.434541,0.292713,0.436906,0.347333,0.376647,0.434945,0.256730,0.339621,0.434945,0.434945,0.340782,0.391786,0.309182,0.262508,0.262508,0.340311,0.305271,0.376647,0.282940,0.258431,0.435120,0.284541,0.435120,0.436906
5d610ae1653a331687083239,0.424553,0.000000,0.244767,0.254179,0.321831,0.362632,0.332871,0.300025,0.362632,0.356453,0.278347,0.426486,0.170655,0.284061,0.296546,0.248505,0.333753,0.399447,0.193195,0.293021,0.287596,0.284337,0.220405,0.268981,0.366166,0.426486,0.295849,0.294617,0.360748,0.330990,0.362632,0.280518,0.243671,0.363202,0.253701,0.283899,0.226396,0.285213,0.288176,0.363202,...,0.243871,0.331112,0.333358,0.363780,0.334760,0.292295,0.215423,0.312653,0.423104,0.181413,0.363202,0.285095,0.293637,0.227075,0.425524,0.366166,0.421620,0.244735,0.364957,0.287302,0.290515,0.369983,0.231547,0.289200,0.369983,0.369983,0.331788,0.366166,0.252291,0.243871,0.243871,0.290080,0.285213,0.290515,0.247160,0.252657,0.369326,0.250990,0.369326,0.364957
5d618359fc5fcf3bdd9a0910,0.226683,0.244767,0.000000,0.239083,0.260394,0.293207,0.233121,0.211908,0.293207,0.238775,0.281282,0.295970,0.169824,0.200411,0.127201,0.185835,0.235262,0.093795,0.158203,0.279938,0.404603,0.232080,0.212062,0.163318,0.301381,0.295970,0.313562,0.169622,0.184137,0.348492,0.293207,0.209544,0.212247,0.454460,0.195396,0.231004,0.129456,0.234197,0.190011,0.294593,...,0.171279,0.24725

In [0]:
def find_n_neighbours(df,n):
  order = np.argsort(df.values, axis = 1)[:, :n]
  df = df.apply(lambda x: pd.Series(x.sort_values(ascending = False).iloc[:n].index, index=['top{}'.format(i) for i in range(1, n+1)]), axis = 1)
  return df

In [0]:
sim_user = find_n_neighbours(similarity_with_users,3)
sim_user.head()

,top1,top2,top3
user_id,,,
5d60098a653a331687083238,5e8447a7a3258347b42f2446,5ebc2ec4514aab59896bcc84,5e5855ced701ab08af792b51
5d610ae1653a331687083239,5e902793a3258347b42f2733,5df11ee1ee4bb5252b4f5344,5e5855ced701ab08af792b51
5d618359fc5fcf3bdd9a0910,5ece02427023451e66223861,5ed232b976027d35905cc689,5e7dde87a3258347b42f2108
5d6d2bb87fa40e1417a49315,5e3563348d344822fed4d13a,5defd51362624b0135ea9fd2,5ec3ba5374f7660d73aa1201
5d7c994d5720533e15c3b1e9,5eb508ae10426255a7aaa12c,5d6d2bb87fa40e1417a49315,5df49b32cc709107827fb3c7


In [0]:
def user_item_score(user):
  a = sim_user[sim_user.index==user].values
  b = a.squeeze().tolist()
  
  final_scores = []
  for i in range(len(posts)):
    row = posts.iloc[i]
    item = row['_id']
    if item not in dic_post_user[user]:
      c = view_matrix.loc[:,item]
      d = c[c.index.isin(b)]
      f = d[d.notnull()]
      index = f.index.values.squeeze().tolist()
      corr = similarity_with_users.loc[user,index]
      
      fin = pd.concat([f, corr], axis = 1)
      fin.columns = ['adg_score','correlation']
      fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'], axis = 1)
      
      nume = fin['score'].sum()
      deno = fin['correlation'].sum()
      
      final_score = (nume/deno)
      final_scores.append([final_score,row])
  
  final_scores.sort(key = lambda x: x[0], reverse = True)
  
  num_rec = 0
  while final_scores[num_rec][0] >= 0.3:
    num_rec += 1
  final_scores = final_scores[:max(num_rec,5)]
  
  view_user = views[views['user_id'] == user]
  view_user = view_user.sort_values(by = ['timestamp'], ascending = False)

  user_posts = posts[posts['_id'].isin(view_user['post_id'])]
  user_posts = pd.merge(user_posts, view_user, left_on = '_id', right_on = 'post_id')
  user_posts = user_posts.sort_values(by = ['timestamp'], ascending = False)

  print('_'*100)
  print('_'*100)
  print()
  print("POSTS ALREADY SEEN BY USER\n")
  for i in range(len(user_posts)):
    post = user_posts.iloc[i]
    print("Title: {}".format(post['title']))
    print("Category: {}".format(post['category']))
    print("Post Type: {}".format(post[' post_type']))
    print()
  
  print('_'*100)
  print('_'*100)
  print()
  print("POSTS RECOMMENDED FOR USER\n")
  for i in range(len(final_scores)):
    post = final_scores[i][1]
    if post['category']!='':
      print("Title: {}".format(post['title']))
      print("Category: {}".format(post['category']))
      print("Post Type: {}".format(post[' post_type']))
      print()

In [0]:
user = input("Enter the user id to whom you want to recommend : ")
print()
user_item_score(user)

Enter the user id to whom you want to recommend : 5ed0defa76027d35905cc2de

____________________________________________________________________________________________________
____________________________________________________________________________________________________

POSTS ALREADY SEEN BY USER

Title: She got a new hairstyle! Photo by Chirag
Category: Photography
Post Type: artwork

Title: AWS services and how to launch OS on AWS Cloud
Category: Computer Technology|Cloud Computing
Post Type: blog

Title: Configure Docker with Django; PostgreSQL; Pg-admin & Elasticsearch
Category: Computer Technology|Computer Application|Information Technology
Post Type: blog

Title: Energy.
Category: Calligraphy|Typography|Pen and ink
Post Type: artwork

Title: Face Recognition using Transfer Learning.
Category: Science; Technology
Post Type: skill

____________________________________________________________________________________________________
___________________________________________